## Rozwiązywanie układów równań liniowych metodami interacyjnymi 

### Zadanie 1
Zaimplementuj metodę Jacobiego. Podaj warunki jej stosowalności. Wygeneruj co najmniej trzy odpowiednie macierze o różnych wielkościach i sprawdź działanie swojej metody. Zwróć uwagę na zbieżność tej metody. 

In [40]:
import numpy as np

def jacobiego(A,B,numberOfIterations):
    x = np.zeros_like(B)
    
    for it_count in range(numberOfIterations):
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
        
        #Kończymy działanie jeżeli kolejne iteracje nie zmieniają wyniku
        if np.allclose(x, x_new, atol=1e-10, rtol=0.):
            return x
        x = x_new


def generateEquation(matrixDegree,maxIterations):
    #Generujemy macierz A
    A = np.zeros((matrixDegree, matrixDegree))
    for i in range(matrixDegree):
        sum = 0
        for j in range(matrixDegree):
            if j >= i: 
                A[i][j] = A[j][i] = np.random.randint(-40,40)
            if i != j: 
                sum += abs(A[i][j])
        A[i][i] = sum
    #Generujemy wektor rozwiązań 
    X=np.random.randint(-15,15,matrixDegree)
    
    #Generujemy wektor wspólczyników wolnych
    B=[]
    for i in range(0,matrixDegree):
        numberToAdd=0
        for j in range(0,matrixDegree):
            numberToAdd+=X[j]*A[i][j]
        B.append(numberToAdd)
    return A,X,B
        
def solveJacobiego(matrixDegree,maxIterations):
    A,X,B=generateEquation(matrixDegree,maxIterations)
    print("Wektor rozwiązań")
    print(X)
    print("Rozwiązania znalezione przy pomocy metody Jacobiego :")
    print(jacobiego(A, B, maxIterations))

solveJacobiego(5,1000)
solveJacobiego(10,1000)
solveJacobiego(14,1000)

Wektor rozwiązań
[-1  5 11 -8  7]
Rozwiązania znalezone przy pomocy metody jacobiego :
[-1.  5. 11. -8.  7.]
Wektor rozwiązań
[  9  -4  -9   6 -11  -8  -4   7 -11 -15]
Rozwiązania znalezone przy pomocy metody jacobiego :
[  9.  -4.  -9.   6. -11.  -8.  -4.   7. -11. -15.]
Wektor rozwiązań
[ -8   7 -11   7 -14  -3   8 -14   7   8  -6   5   2  -3]
Rozwiązania znalezone przy pomocy metody jacobiego :
[ -8.   7. -11.   7. -14.  -3.   8. -14.   7.   8.  -6.   5.   2.  -3.]


**Metoda Jacobiego:** 

**stosujemy dla A**  
* silnie diagonalnie dominujących.  

**zbieżna dla A**  
* nieredukowalnych,  
* silnie diagonalnie dominujących.

### Zadanie 2
Zaimplementuj metodę Gaussa-Seidla i kolejnych nadrelaksacji (successive over-relaxation). Podaj warunki stosowalności tych metod. Przeprowadź badanie działania swoich implementacji analogicznie jak w poprzednim zadaniu. Porównaj zbieżność wszystkich trzech metod. 

In [54]:
import numpy as np
def gaussaSeidla(A,B,numberOfIterations):
    x = np.zeros_like(B)
    
    for it_count in range(numberOfIterations):
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
            
        if np.allclose(x, x_new, rtol=1e-8):
            return x
        x = x_new


def overRelaxation(A, B, omega,numberOfIterations):
    x_new = np.zeros_like(B)
    residual = np.linalg.norm(np.matmul(A, x_new) - B) #Initializacja 
    for k in range(numberOfIterations):
        for i in range(A.shape[0]):
            sigma = 0
            for j in range(A.shape[1]):
                if j != i:
                    sigma += A[i][j] * x_new[j]
            x_new[i] = (1 - omega) * x_new[i] + (omega / A[i][i]) * (B[i] - sigma)
        residual = np.linalg.norm(np.matmul(A, x_new) - B)
        if residual < 1e-8:
            return x_new
    return x_new

def solveGSOR(matrixDegree,maxIterations,omega):
    A,X,B=generateEquation(matrixDegree,maxIterations)
    print("Wektor rozwiązań")
    print(X)
    print("Rozwiązania znalezione przy pomocy metody Gaussa-Seidla :")
    print(gaussaSeidla(A, B, maxIterations))
    print("Rozwiązania znalezione przy pomocy kolejnych nadrelaksacji:")
    print(overRelaxation(A,B,omega,maxIterations))
    
solveGSOR(5,1000,0.5)
solveGSOR(10,1000,0.5)
solveGSOR(14,1000,0.5)

Wektor rozwiązań
[ 2  1  8 -8 -5]
Rozwiązania znalezone przy pomocy metody Gaussa-Seidla :
[ 2.00000003  0.99999998  8.00000001 -7.99999998 -4.99999999]
Rozwiązania znalezone przy pomocy  kolejnych nadrelaksacji:
[ 2.  1.  8. -8. -5.]
Wektor rozwiązań
[  4  13  -2 -10  -8  12  -9 -13 -13  12]
Rozwiązania znalezone przy pomocy metody Gaussa-Seidla :
[  4.          12.99999996  -2.00000003  -9.99999997  -7.99999997
  12.00000001  -8.99999999 -12.99999997 -13.00000002  11.99999999]
Rozwiązania znalezone przy pomocy  kolejnych nadrelaksacji:
[  4.  13.  -2. -10.  -8.  12.  -9. -13. -13.  12.]
Wektor rozwiązań
[-12  11   3 -14   1   5   2  -2  10  -8   9   9 -14   4]
Rozwiązania znalezone przy pomocy metody Gaussa-Seidla :
[-12.00000002  11.00000002   3.00000001 -13.99999998   1.00000002
   4.99999999   2.00000003  -2.00000001   9.99999999  -7.99999998
   8.99999999   9.         -14.00000001   3.99999999]
Rozwiązania znalezone przy pomocy  kolejnych nadrelaksacji:
[-12.  11.   3. -14.   1. 

**Metoda Gaussa Seidla:**
 
**zbieżna dla A:**  
* silnie diagonalnie dominujących wierszowo, kolumnowo,  
* symetrycznych,  
* dodatnio określonych  

**Metoda kolejnych nadrelaksacji** - jest zmodyfikowaną metodą Gaussa-Seidela 

**zbieżna dla A:**
* kiedy parametr omega należy do przedziału (0, 2)

In [64]:
def jacobiegoI(A,B,numberOfIterations):
    x = np.zeros_like(B)
    
    for it_count in range(numberOfIterations):
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
        
        #Kończymy działanie jeżeli kolejne iteracje nie zmieniają wyniku
        if np.allclose(x, x_new, atol=1e-10, rtol=0.):
            return it_count
        x = x_new

def gaussaSeidlaI(A,B,numberOfIterations):
    x = np.zeros_like(B)
    
    for it_count in range(numberOfIterations):
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
        if np.allclose(x, x_new, rtol=1e-8):
            return it_count
        x = x_new


def overRelaxationI(A, B, omega,numberOfIterations):
    x_new = np.zeros_like(B)
    residual = np.linalg.norm(np.matmul(A, x_new) - B) #Initializacja 
    for k in range(numberOfIterations):
        for i in range(A.shape[0]):
            sigma = 0
            for j in range(A.shape[1]):
                if j != i:
                    sigma += A[i][j] * x_new[j]
            x_new[i] = (1 - omega) * x_new[i] + (omega / A[i][i]) * (B[i] - sigma)
        residual = np.linalg.norm(np.matmul(A, x_new) - B)
        if residual < 1e-8:
            return k
    return numberOfIterations

def compareIterations(matrixDegree,maxIterations,omega):
    A,X,B=generateEquation(matrixDegree,maxIterations)
    print("Ilość iteracji dla macierzy stopnia",matrixDegree,":")
    print("\tMetoda Jacobiego" ,jacobiegoI(A,B,maxIterations))
    print("\tMetoda Gaussa-Seidla ",gaussaSeidlaI(A, B, maxIterations))
    print("\tMetoda kolejnych nadrelaksacji ",overRelaxationI(A,B,omega,maxIterations))
    
compareIterations(5,1000,0.5)
compareIterations(50,1000,0.5)
compareIterations(100,1000,0.5)


Ilość iteracji dla macierzy stopnia 5 :
	Metoda Jacobiego 105
	Metoda Gaussa-Seidla  28
	Metoda kolejnych nadrelaksacji  131
Ilość iteracji dla macierzy stopnia 50 :
	Metoda Jacobiego 21
	Metoda Gaussa-Seidla  9
	Metoda kolejnych nadrelaksacji  56
Ilość iteracji dla macierzy stopnia 100 :
	Metoda Jacobiego 17
	Metoda Gaussa-Seidla  8
	Metoda kolejnych nadrelaksacji  54
